In [3]:
# Using ! to invoke the following shell command inside our notebook
!git clone https://bitbucket.org/jadslim/german-traffic-signs

Cloning into 'german-traffic-signs'...
Unpacking objects: 100% (6/6), done.


In [5]:
# The above cell cloned the data from the bitbucket server and created a folder
# called "german-traffic-sign". Now we list the data inside this folder
!ls german-traffic-signs/

signnames.csv  test.p  train.p	valid.p


As it can be seen there are 4 files. First one is a spreadsheet of sign name, and the other 3 files are pickle files, that contain our respective training, test and validation datasets.

In python to save something on disk, it can be pickled. That is it can be serialized before writing it to file. By serializing it, it converts all the object to a character stream.

Pickled file contain serialized data that can be unpickled when desired

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [2]:
np.random.seed(0)